In [1]:
from ibm_watsonx_ai import Credentials
from ibm_watsonx_ai.foundation_models import ModelInference

In [8]:
from pprint import pprint

In [2]:
credentials = Credentials(
    url="https://us-south.ml.cloud.ibm.com",
    api_key="AnzfgthfcrfRzttoXGiKZUJDMRlcB3w4uemf0PJGFFT5"
)

In [ ]:
model = ModelInference(
    model_id="mistralai/mistral-large", # Che conosciamo bene 😊🏆 
    credentials=credentials,
    project_id="5c33debb-5a25-4bfe-8392-ede4b20884fe",
    params={
        "max_tokens": 200
      }
)

result = model.chat()

print(result['choices'][0]['message']['content'])

In [ ]:
# bring in our LLAMA_CLOUD_API_KEY
from dotenv import load_dotenv
load_dotenv()

# bring in deps
from llama_parse import LlamaParse
from llama_index.core import SimpleDirectoryReader

# set up parser
parser = LlamaParse(
    result_type="text"  # "markdown" and "text" are available
)

filename = "./data/Menu/Le Dimensioni del Gusto.pdf"

# use SimpleDirectoryReader to parse our file
file_extractor = {".pdf": parser}
documents = SimpleDirectoryReader(input_files=[filename], file_extractor=file_extractor).load_data()
print(documents)

In [ ]:
doc_text = ""
for i in range(0, len(documents)):
    doc_text += documents[i].text

In [ ]:
documents

In [ ]:
documents[0].metadata['file_name']

In [71]:
def normalize_whitespace(text):
        import re
        """
        Reduces multiple consecutive whitespace characters to a single space.

        Args:
        - text (str): The input text with excessive whitespace.

        Returns:
        - str: The text with reduced whitespace.
        """
        # Replace one or more whitespace characters with a single space
        return re.sub(r'\s+', ' ', text).strip()

In [ ]:
import pandas as pd

# Ensure both elements are lists (or iterable)
data = [
    [documents[0].metadata['file_name'], doc_text]
]

df = pd.DataFrame(columns=["FileName", "Text"])
# New row data as a DataFrame
new_row_df = pd.DataFrame(data, columns=["FileName", "Text"])

# Using pd.concat to add the new row
df = pd.concat([df, new_row_df], ignore_index=True)

print(df)


In [ ]:
pprint(df.Text[0])